# Multi-Lingual Search with Amazon Nova Multimodal Embeddings

Building search that works across languages is tricky. When someone searches in English for "set alarm," you want to find relevant results even if your content is in Spanish ("pon alarma") or French ("règle l'alarme"). Traditional keyword matching fails here because the words are completely different.

You'll build a search system that understands meaning instead of just matching words. Amazon Nova Multimodal Embeddings converts text into numbers (vectors) that capture semantic meaning. Text with similar meanings gets similar numbers, regardless of language.

The system works by converting multilingual text to 1024-dimensional vectors using Nova, storing them in S3 Vectors for fast similarity search, then returning ranked results based on cosine similarity. We'll test it with voice assistant commands in multiple languages and measure how well it matches across language boundaries.

Before you start, make sure you have an AWS account with Bedrock access in us-east-1, Python 3.8+, and basic understanding of embeddings.

## 1. Environment Setup

First, you need to install the packages and import libraries. The key ones are boto3 for talking to AWS services, scikit-learn for calculating similarity between vectors, pandas and numpy for handling data, and matplotlib for creating charts. The tqdm library shows progress bars, which is helpful since embedding generation can take a while.

The code below installs everything you need and sets up plotting with a nice color scheme that works well when you have multiple languages in your visualizations.

In [ ]:
# Install required packages
!pip install boto3 pandas numpy scikit-learn matplotlib seaborn tqdm langdetect

import boto3
import json
import pandas as pd
import numpy as np
import os
import glob
from typing import List, Dict, Optional, Tuple
import base64
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)

print('✅ Libraries imported successfully!')

## 2. AWS Configuration

Now you'll set up connections to the AWS services. You need three clients: Bedrock Runtime to call Nova for generating embeddings, S3 Vectors to store and search your vectors, and regular S3 for any extra file storage.

Nova is currently only available in us-east-1, so that's where everything needs to run. The bucket name includes a hash to avoid naming conflicts with other AWS accounts. The embedding dimension of 1024 gives you a good balance between accuracy and performance.

This configuration block creates all the AWS clients and prints out your settings so you can verify everything looks right.

In [ ]:
# AWS Configuration
AWS_REGION = 'us-east-1'  # Nova Multimodal Embeddings region
S3_BUCKET_NAME = 'nova-multilingual-search-' + str(hash(os.getlogin()))[-6:]  # Unique bucket
DATASET_PATH = './massive'  # Path to MASSIVE dataset

# Initialize AWS clients
try:
    bedrock_client = boto3.client('bedrock-runtime', region_name=AWS_REGION)
    s3_client = boto3.client('s3', region_name=AWS_REGION)
    s3vectors_client = boto3.client('s3vectors', region_name=AWS_REGION)
    print('✅ AWS clients initialized successfully!')
except Exception as e:
    print(f'❌ AWS client initialization failed: {e}')
    print('Please ensure AWS credentials are configured')

# Model configurations
NOVA_MODEL_ID = 'amazon.nova-2-multimodal-embeddings-v1:0'
EMBEDDING_DIMENSION = 1024  # Balanced performance

print(f'🌍 AWS Region: {AWS_REGION}')
print(f'🪣 S3 Bucket: {S3_BUCKET_NAME}')
print(f'🤖 Nova Model: {NOVA_MODEL_ID}')
print(f'📏 Embedding Dimension: {EMBEDDING_DIMENSION}')

## 2.1. Dataset Configuration

You can choose between two data sources depending on whether you want to experiment quickly or test with real data. The MASSIVE dataset contains over a million voice assistant commands in 52 languages, with each command labeled by intent. It's perfect for seeing how the system performs at scale, but remember that more data means more API calls to Nova.

The sample data option creates just 12 examples across 4 languages with common voice commands like setting alarms and playing music. This is great for testing your setup without worrying about costs.

The configuration below lets you switch between modes easily. If you choose MASSIVE, the code will download and extract it automatically.

In [ ]:
# Dataset Configuration
USE_MASSIVE_DATASET = True  # Set to True to use full MASSIVE dataset, False for sample data

# MASSIVE dataset configuration
MASSIVE_URL = 'https://amazon-massive-nlu-dataset.s3.amazonaws.com/amazon-massive-dataset-1.0.tar.gz'
MASSIVE_ARCHIVE = 'amazon-massive-dataset-1.0.tar.gz'
MASSIVE_DATA_DIR = '1.0/data'

# Sample languages to use (when using MASSIVE dataset)
SAMPLE_LANGUAGES = ['en-US', 'es-ES', 'fr-FR', 'de-DE']  # Can expand to all 52 languages

print(f'📊 Dataset Mode: {"MASSIVE" if USE_MASSIVE_DATASET else "Sample"}')
if USE_MASSIVE_DATASET:
    print(f'🌍 Languages: {len(SAMPLE_LANGUAGES)} selected from 52 available')
else:
    print(f'🌍 Languages: 4 sample languages (en-US, es-ES, fr-FR, de-DE)')

## 3. Core System Classes

The search system needs three main components working together. The NovaEmbeddings class handles all communication with the Nova API, including error handling and cost tracking. The S3VectorStore class manages your vector database, automatically creating the necessary AWS resources and handling batch uploads. The MultilingualSearchEngine class ties everything together, providing a simple interface for converting queries to vectors and finding similar content.

These classes abstract away the complexity of AWS service interactions so you can focus on building your search logic. The code below defines all three classes with proper error handling and progress tracking.



In [ ]:
class NovaEmbeddings:
    """Client for Amazon Nova Multimodal Embeddings"""
    
    def __init__(self, region='us-east-1'):
        self.bedrock = boto3.client('bedrock-runtime', region_name=region)
        self.model_id = 'amazon.nova-2-multimodal-embeddings-v1:0'
        self.request_count = 0
    
    def embed_text(self, text: str, dimension: int = 1024) -> List[float]:
        if not text or not text.strip():
            return [0.0] * dimension
            
        request_body = {
            'taskType': 'SINGLE_EMBEDDING',
            'singleEmbeddingParams': {
                'embeddingDimension': dimension,
                'embeddingPurpose': 'GENERIC_INDEX',
                'text': {"truncationMode": "END", 'value': text[:8000]}
            }
        }
        
        try:
            response = self.bedrock.invoke_model(
                modelId=self.model_id,
                body=json.dumps(request_body)
            )
            result = json.loads(response['body'].read())
            self.request_count += 1
            return result['embeddings'][0]['embedding']
        except Exception as e:
            print(f'Error embedding text: {e}')
            return [0.0] * dimension
    
    def embed_batch_text(self, texts: List[str], dimension: int = 1024, batch_size: int = 5) -> List[List[float]]:
        embeddings = []
        for text in tqdm(texts, desc='Generating embeddings'):
            embedding = self.embed_text(text, dimension)
            embeddings.append(embedding)
        return embeddings


In [ ]:

class S3VectorStore:
    """Amazon S3 Vectors-based vector store"""
    
    def __init__(self, vector_bucket_name: str, index_name: str, dimension: int = 1024, region: str = 'us-east-1'):
        self.s3vectors = boto3.client('s3vectors', region_name=region)
        self.vector_bucket_name = vector_bucket_name
        self.index_name = index_name
        self.dimension = dimension
        self._setup_vector_store()
    
    def _setup_vector_store(self):
        # Create vector bucket with encryption
        try:
            self.s3vectors.get_vector_bucket(vectorBucketName=self.vector_bucket_name)
            print(f'✅ Vector bucket {self.vector_bucket_name} exists')
        except self.s3vectors.exceptions.NotFoundException:
            self.s3vectors.create_vector_bucket(
                vectorBucketName=self.vector_bucket_name,
                encryptionConfiguration={'sseType': 'AES256'}  # SSE-S3 encryption
            )
            print(f'✅ Created vector bucket: {self.vector_bucket_name} (encrypted with SSE-S3)')
        
        # Create vector index
        try:
            self.s3vectors.get_index(vectorBucketName=self.vector_bucket_name, indexName=self.index_name)
            print(f'✅ Vector index {self.index_name} exists')
        except self.s3vectors.exceptions.NotFoundException:
            self.s3vectors.create_index(
                vectorBucketName=self.vector_bucket_name,
                indexName=self.index_name,
                dimension=self.dimension,
                dataType='float32',
                distanceMetric='cosine'
            )
            print(f'✅ Created index: {self.index_name}')
    
    def add_embeddings(self, embeddings: List[List[float]], metadata: List[Dict], batch_size: int = 500):
        vectors = []
        for i, (embedding, meta) in enumerate(zip(embeddings, metadata)):
            vectors.append({
                'key': f"{meta.get('locale', 'unknown')}:{meta.get('id', i)}",
                'data': {'float32': embedding},
                'metadata': {
                    'intent': meta.get('intent', 'unknown'),
                    'locale': meta.get('locale', 'unknown'),
                    'text': meta.get('text', '')[:500],
                    'scenario': meta.get('scenario', 'unknown')
                }
            })
        
        # Add in batches
        for i in tqdm(range(0, len(vectors), batch_size), desc='Adding to S3 Vectors'):
            batch = vectors[i:i + batch_size]
            try:
                self.s3vectors.put_vectors(
                    vectorBucketName=self.vector_bucket_name,
                    indexName=self.index_name,
                    vectors=batch
                )
            except Exception as e:
                print(f'Error adding batch: {e}')
        
        print(f'✅ Added {len(vectors)} vectors to S3 Vectors')
    
    def search(self, query_embedding: List[float], k: int = 10, metadata_filter: Optional[Dict] = None) -> List[Dict]:
        try:
            params = {
                'vectorBucketName': self.vector_bucket_name,
                'indexName': self.index_name,
                'queryVector': {'float32': query_embedding},
                'topK': k,
                'returnDistance': True,
                'returnMetadata': True
            }
            
            if metadata_filter:
                params['metadataFilter'] = metadata_filter
            
            response = self.s3vectors.query_vectors(**params)
            
            results = []
            for i, vector in enumerate(response.get('vectors', [])):
                result = {
                    'key': vector['key'],
                    'similarity_score': 1.0 - vector['distance'],
                    'rank': i + 1
                }
                if vector.get('metadata'):
                    result.update(vector['metadata'])
                results.append(result)
            
            return results
        except Exception as e:
            print(f'Search error: {e}')
            return []



In [ ]:

class MultilingualSearchEngine:
    """Search engine using S3 Vectors"""
    
    def __init__(self, embeddings_client, vector_store: S3VectorStore):
        self.embeddings = embeddings_client
        self.vector_store = vector_store
        self.search_history = []
    
    def search(self, query: str, k: int = 10, target_languages: Optional[List[str]] = None) -> List[Dict]:
        query_embedding = self.embeddings.embed_text(query)
        
        metadata_filter = None
        if target_languages:
            metadata_filter = {'locale': {'in': target_languages}}
        
        results = self.vector_store.search(query_embedding, k, metadata_filter)
        
        self.search_history.append({
            'query': query,
            'target_languages': target_languages,
            'results_count': len(results)
        })
        
        return results
    
    def semantic_similarity_analysis(self, text1: str, text2: str) -> Dict:
        emb1 = self.embeddings.embed_text(text1)
        emb2 = self.embeddings.embed_text(text2)
        similarity = cosine_similarity([emb1], [emb2])[0][0]
        
        return {
            'text1': text1,
            'text2': text2,
            'cosine_similarity': float(similarity),
            'semantic_distance': float(1 - similarity)
        }

## 4. Data Loading

The data loading process handles both the large MASSIVE dataset and the small sample dataset using the same structure. If you chose MASSIVE, the code downloads the compressed archive, extracts the JSONL files organized by language, and loads the voice commands with their labels. The max_samples_per_lang parameter lets you limit how much data you load to control API costs during development.

For the sample data option, the code creates a small dataset with common voice commands translated into four languages. Each command has the same intent across languages, so "set alarm" in English corresponds to "pon alarma" in Spanish.

Both approaches create pandas DataFrames with identical column structures, so you can switch between them without changing any downstream code. The functions below handle all the downloading, extraction, and data formatting automatically.

In [ ]:
import tarfile
import urllib.request

def download_massive_dataset():
    """Download and extract MASSIVE dataset"""
    if not os.path.exists(MASSIVE_ARCHIVE):
        print(f'📥 Downloading MASSIVE dataset from {MASSIVE_URL}...')
        urllib.request.urlretrieve(MASSIVE_URL, MASSIVE_ARCHIVE)
        print('✅ Download complete')
    
    if not os.path.exists(MASSIVE_DATA_DIR):
        print('📦 Extracting dataset...')
        with tarfile.open(MASSIVE_ARCHIVE, 'r:gz') as tar:
            tar.extractall()
        print('✅ Extraction complete')

def load_massive_data(languages=None, max_samples_per_lang=1000):
    """Load MASSIVE dataset from JSONL files"""
    if languages is None:
        languages = SAMPLE_LANGUAGES
    
    multilingual_data = {}
    
    for lang in languages:
        jsonl_path = os.path.join(MASSIVE_DATA_DIR, f'{lang}.jsonl')
        if os.path.exists(jsonl_path):
            data = []
            with open(jsonl_path, 'r', encoding='utf-8') as f:
                for i, line in enumerate(f):
                    if i >= max_samples_per_lang:
                        break
                    data.append(json.loads(line.strip()))
            multilingual_data[lang] = pd.DataFrame(data)
            print(f'  {lang}: {len(data)} samples loaded')
        else:
            print(f'  ⚠️  {lang}: File not found')
    
    return multilingual_data

def create_sample_data():
    """Create sample data in MASSIVE format and save as JSONL files"""
    sample_data = {
        'en-US': [
            {'id': '1', 'locale': 'en-US', 'partition': 'train', 'intent': 'alarm_set', 'scenario': 'alarm', 'utt': 'set alarm for 7 AM tomorrow', 'annot_utt': 'set alarm for [time : 7 AM] [date : tomorrow]', 'worker_id': '1'},
            {'id': '2', 'locale': 'en-US', 'partition': 'train', 'intent': 'weather_query', 'scenario': 'weather', 'utt': 'what is the weather like today', 'annot_utt': 'what is the weather like [date : today]', 'worker_id': '1'},
            {'id': '3', 'locale': 'en-US', 'partition': 'train', 'intent': 'music_play', 'scenario': 'music', 'utt': 'play some jazz music', 'annot_utt': 'play some [music_genre : jazz] music', 'worker_id': '1'},
        ],
        'es-ES': [
            {'id': '1', 'locale': 'es-ES', 'partition': 'train', 'intent': 'alarm_set', 'scenario': 'alarm', 'utt': 'pon la alarma para las 7 de la mañana', 'annot_utt': 'pon la alarma para las [time : 7 de la mañana]', 'worker_id': '1'},
            {'id': '2', 'locale': 'es-ES', 'partition': 'train', 'intent': 'weather_query', 'scenario': 'weather', 'utt': '¿cómo está el tiempo hoy?', 'annot_utt': '¿cómo está el tiempo [date : hoy]?', 'worker_id': '1'},
            {'id': '3', 'locale': 'es-ES', 'partition': 'train', 'intent': 'music_play', 'scenario': 'music', 'utt': 'reproduce música jazz', 'annot_utt': 'reproduce música [music_genre : jazz]', 'worker_id': '1'},
        ],
        'fr-FR': [
            {'id': '1', 'locale': 'fr-FR', 'partition': 'train', 'intent': 'alarm_set', 'scenario': 'alarm', 'utt': 'règle l\'alarme pour 7 heures demain matin', 'annot_utt': 'règle l\'alarme pour [time : 7 heures] [date : demain matin]', 'worker_id': '1'},
            {'id': '2', 'locale': 'fr-FR', 'partition': 'train', 'intent': 'weather_query', 'scenario': 'weather', 'utt': 'quel temps fait-il aujourd\'hui', 'annot_utt': 'quel temps fait-il [date : aujourd\'hui]', 'worker_id': '1'},
            {'id': '3', 'locale': 'fr-FR', 'partition': 'train', 'intent': 'music_play', 'scenario': 'music', 'utt': 'joue de la musique jazz', 'annot_utt': 'joue de la musique [music_genre : jazz]', 'worker_id': '1'},
        ],
        'de-DE': [
            {'id': '1', 'locale': 'de-DE', 'partition': 'train', 'intent': 'alarm_set', 'scenario': 'alarm', 'utt': 'stelle den Wecker auf 7 Uhr morgen früh', 'annot_utt': 'stelle den Wecker auf [time : 7 Uhr] [date : morgen früh]', 'worker_id': '1'},
            {'id': '2', 'locale': 'de-DE', 'partition': 'train', 'intent': 'weather_query', 'scenario': 'weather', 'utt': 'wie ist das Wetter heute', 'annot_utt': 'wie ist das Wetter [date : heute]', 'worker_id': '1'},
            {'id': '3', 'locale': 'de-DE', 'partition': 'train', 'intent': 'music_play', 'scenario': 'music', 'utt': 'spiele Jazz-Musik', 'annot_utt': 'spiele [music_genre : Jazz]-Musik', 'worker_id': '1'},
        ]
    }
    
    # Create directory structure matching MASSIVE format
    sample_data_dir = '1.0/data'
    os.makedirs(sample_data_dir, exist_ok=True)
    
    # Save as JSONL files
    for lang, samples in sample_data.items():
        jsonl_path = os.path.join(sample_data_dir, f'{lang}.jsonl')
        with open(jsonl_path, 'w', encoding='utf-8') as f:
            for sample in samples:
                f.write(json.dumps(sample, ensure_ascii=False) + '\n')
        print(f'  📄 Created {jsonl_path}')
    
    # Load back as DataFrames for consistency
    multilingual_data = {}
    for lang, samples in sample_data.items():
        multilingual_data[lang] = pd.DataFrame(samples)
    
    return multilingual_data

# Load data based on configuration
if USE_MASSIVE_DATASET:
    print('📊 Loading MASSIVE dataset...')
    download_massive_dataset()
    multilingual_data = load_massive_data(SAMPLE_LANGUAGES, max_samples_per_lang=100)  # Limit for demo
    print(f'✅ Loaded MASSIVE data for {len(multilingual_data)} languages')
else:
    print('📊 Creating sample data...')
    print('📁 Creating MASSIVE-compatible directory structure...')
    multilingual_data = create_sample_data()
    print(f'✅ Created sample data for {len(multilingual_data)} languages')
    print('📁 Sample JSONL files created in 1.0/data/ directory')

# Display summary
total_samples = sum(len(df) for df in multilingual_data.values())
print(f'\n📈 Dataset Summary:')
for lang, df in multilingual_data.items():
    print(f'  {lang}: {len(df)} samples')
print(f'  Total: {total_samples} samples')

## 5. System Initialization

With your classes defined and data loaded, you can now initialize the complete search system. The process starts by creating a Nova embeddings client that connects to Bedrock in your specified region. This client handles all the embedding generation requests with automatic retry logic.

Next, you create the S3 Vector Store, which automatically provisions the necessary AWS infrastructure. It creates an encrypted vector bucket and sets up a cosine similarity index optimized for Nova's 1024-dimensional output. If these resources already exist, the code detects them and uses the existing setup.

Finally, the search engine combines both components into a single interface that handles the complete workflow from query to results. The initialization code below sets up all three components and validates that your AWS credentials are working properly.

In [ ]:
# Initialize system components
print('🚀 Initializing Nova Multilingual Search System...')

# Initialize Nova embeddings client
nova_embeddings = NovaEmbeddings(region=AWS_REGION)

# Initialize S3 Vector Store
vector_store = S3VectorStore(
    vector_bucket_name=S3_BUCKET_NAME + '-vectors',
    index_name='multilingual-embeddings',
    dimension=EMBEDDING_DIMENSION,
    region=AWS_REGION
)

# Initialize search engine
search_engine = MultilingualSearchEngine(nova_embeddings, vector_store)

print('✅ System initialized successfully!')

## 6. Embedding Generation & Storage

Now comes the core transformation where your multilingual text becomes searchable vectors. The process starts by extracting all the text and metadata from your loaded datasets. Each piece of text gets paired with information like its language, intent classification, and scenario context.

The embedding generation step sends each text string to Nova, which converts it into a 1024-dimensional vector that captures semantic meaning. The magic happens here - Nova's training on multilingual data means that "set alarm" in English and "pon alarma" in Spanish get very similar vector representations, even though the words are completely different.

Once you have all the vectors, they get uploaded to S3 Vectors along with their metadata. Each vector is indexed with a key that combines the language and sample ID, making lookups efficient. The metadata travels with each vector, so you can filter searches by language or intent later.

The code below handles this entire pipeline with progress tracking, since generating embeddings for large datasets can take several minutes.

In [ ]:
# Prepare data for embedding
all_texts = []
all_metadata = []

for language, df in multilingual_data.items():
    for _, row in df.iterrows():
        all_texts.append(row['utt'])
        all_metadata.append({
            'id': row['id'],
            'locale': language,
            'intent': row['intent'],
            'scenario': row['scenario'],
            'text': row['utt']
        })

print(f'📝 Prepared {len(all_texts)} texts for embedding')

# Generate embeddings
print('🔄 Generating Nova embeddings...')
embeddings = nova_embeddings.embed_batch_text(all_texts, dimension=EMBEDDING_DIMENSION)

# Add to vector store
print('📦 Adding embeddings to S3 Vectors...')
vector_store.add_embeddings(embeddings, all_metadata)

print('✅ Embeddings generated and stored successfully!')

## 7. Cross-lingual Search Demonstration

Time to see if your multilingual search actually works. The demonstration takes a sample of your data and uses each utterance as a search query to see what the system finds. You'll sample about 30% of your available utterances, capped at 50 to keep API costs reasonable.

For each query, the system converts the text to a vector using Nova, then searches your indexed corpus for the most similar vectors using cosine similarity. The top 3 results show whether the system can find semantically equivalent content in different languages.

What you want to see is queries in one language returning results with the same intent in other languages. An English query about "set alarm" should retrieve Spanish "pon alarma", French "règle l'alarme", and German "stelle Wecker" with high similarity scores above 0.8.

The search demonstration below runs through your sample queries and displays the results, giving you a clear picture of how well Nova captures cross-lingual semantic meaning.

In [ ]:
# Sample 30% of dataset for cross-lingual search, capped at 50 queries
import random
random.seed(42)  # For reproducible results

all_samples = []
for lang, df in multilingual_data.items():
    for _, row in df.iterrows():
        all_samples.append({'query': row['utt'], 'language': lang, 'intent': row['intent']})

# Sample 30% of data, capped at 50
sample_size = min(50, max(1, int(len(all_samples) * 0.3)))
test_queries = random.sample(all_samples, sample_size)

print(f'📊 Selected {len(test_queries)} queries from {len(all_samples)} total samples')

print('🔍 Cross-lingual Search Demonstration\n')
print('=' * 60)

for i, test_query in enumerate(test_queries, 1):
    query = test_query['query']
    lang = test_query['language']
    intent = test_query['intent']
    
    print(f'\n🌍 Query {i}: "{query}" ({lang}, {intent})')
    print('-' * 40)
    
    results = search_engine.search(query, k=3)
    
    if results:
        for j, result in enumerate(results, 1):
            print(f'  {j}. [{result["locale"]}] {result["text"]}')
            print(f'     Intent: {result["intent"]} | Similarity: {result["similarity_score"]:.4f}')
    else:
        print('  No results found')

print('\n' + '=' * 60)

## 8. Semantic Similarity Analysis

Beyond just seeing search results, you want to measure how well the system captures cross-lingual meaning with hard numbers. This analysis compares utterances that express the same intent in different languages by measuring the cosine similarity between their embedding vectors.

Cosine similarity measures the angle between two vectors in high-dimensional space, giving you a score from 0 to 1. Scores close to 1 mean the vectors point in nearly the same direction, indicating strong semantic alignment despite different surface words.

The analysis picks pairs of utterances with matching IDs across languages - these represent the same intent expressed in different linguistic forms. This controlled comparison isolates Nova's cross-lingual understanding from other semantic variations.

Good multilingual models typically show average similarities above 0.75 for equivalent utterances, with consistent scores across language pairs. You might see higher similarities between structurally similar languages like Spanish and French compared to more distant pairs like English and German.

The code below runs this analysis on 10 cross-lingual pairs and calculates summary statistics to validate your embedding space.

In [ ]:
# Extract 10 cross-lingual pairs using matching IDs
id_groups = {}
for lang, df in multilingual_data.items():
    for _, row in df.iterrows():
        id_key = row['id']
        if id_key not in id_groups:
            id_groups[id_key] = {}
        id_groups[id_key][lang] = row['utt']

test_pairs = []
for id_key, lang_texts in id_groups.items():
    if len(lang_texts) >= 2:
        langs = list(lang_texts.keys())
        for i in range(len(langs)):
            for j in range(i+1, len(langs)):
                test_pairs.append((lang_texts[langs[i]], lang_texts[langs[j]]))

test_pairs = random.sample(test_pairs, min(10, len(test_pairs)))

print(f'🔗 Selected {len(test_pairs)} cross-lingual pairs for similarity analysis')

print('🔗 Cross-lingual Semantic Similarity Analysis\n')

similarities = []
for text1, text2 in test_pairs:
    similarity_result = search_engine.semantic_similarity_analysis(text1, text2)
    similarities.append(similarity_result['cosine_similarity'])
    
    print(f'Text 1: "{text1}"')
    print(f'Text 2: "{text2}"')
    print(f'Cosine Similarity: {similarity_result["cosine_similarity"]:.4f}')
    print('-' * 50)

print(f'📊 Average cross-lingual similarity: {np.mean(similarities):.4f}')
print(f'📈 Similarity range: {min(similarities):.4f} - {max(similarities):.4f}')

## 9. System Evaluation

The final step measures your search system's performance using standard information retrieval metrics. The evaluation focuses on intent-based accuracy - how often the system returns utterances with matching intent classifications from different languages as the top result. This directly tests cross-lingual semantic understanding rather than just lexical matching.

The evaluation runs through your entire dataset, using each utterance as a query and checking whether the top results have the same intent in different languages. It calculates several key metrics: Precision@K measures how many of the top K results are actually relevant, Recall@K shows whether you found all the relevant results within the top K, and Mean Reciprocal Rank tells you the average position of the first good result.

The system also tracks confidence thresholding - results with similarity scores above 0.7 are considered high-confidence matches, which is useful for production deployments where precision matters more than recall. Additionally, it monitors Nova API usage so you can track costs and optimize batch sizes.

For production systems, you typically want to see accuracy above 80% with Mean Reciprocal Rank above 0.85 for acceptable user experience. The evaluation code below runs this complete analysis and provides detailed performance metrics.

In [ ]:
# Intent-based evaluation with confidence thresholds and IR metrics
def evaluate_search_accuracy(search_engine, test_data, confidence_threshold=0.7):
    metrics = {
        'correct_matches': 0,
        'total_queries': 0,
        'precision_at_1': [],
        'precision_at_3': [],
        'recall_at_5': [],
        'mrr': [],
        'high_confidence_matches': 0
    }
    
    for lang, df in test_data.items():
        for _, row in tqdm(df.iterrows(), desc="Running evaluation"):
            query = row['utt']
            query_intent = row['intent']
            query_lang = row['locale']
            
            results = search_engine.search(query, k=5)
            
            # Find cross-lingual matches by intent
            relevant_positions = []
            for i, result in enumerate(results):
                result_lang = result.get('locale', '')
                result_intent = result.get('intent', '')
                similarity = result.get('similarity_score', 0)
                
                # Match: same intent, different language, above threshold
                if result_intent == query_intent and result_lang != query_lang:
                    relevant_positions.append(i + 1)
                    if i == 0:  # First match
                        metrics['correct_matches'] += 1
                    if similarity >= confidence_threshold:
                        metrics['high_confidence_matches'] += 1
            
            # Calculate metrics
            metrics['precision_at_1'].append(1 if relevant_positions and relevant_positions[0] == 1 else 0)
            metrics['precision_at_3'].append(len([p for p in relevant_positions if p <= 3]) / 3)
            metrics['recall_at_5'].append(1 if relevant_positions else 0)
            metrics['mrr'].append(1 / relevant_positions[0] if relevant_positions else 0)
            
            metrics['total_queries'] += 1
    
    # Aggregate metrics
    accuracy = metrics['correct_matches'] / metrics['total_queries'] if metrics['total_queries'] > 0 else 0
    avg_precision_at_1 = np.mean(metrics['precision_at_1'])
    avg_precision_at_3 = np.mean(metrics['precision_at_3'])
    avg_recall_at_5 = np.mean(metrics['recall_at_5'])
    avg_mrr = np.mean(metrics['mrr'])
    
    return {
        'accuracy': accuracy,
        'correct_matches': metrics['correct_matches'],
        'total_queries': metrics['total_queries'],
        'high_confidence_matches': metrics['high_confidence_matches'],
        'precision_at_1': avg_precision_at_1,
        'precision_at_3': avg_precision_at_3,
        'recall_at_5': avg_recall_at_5,
        'mrr': avg_mrr,
        'confidence_threshold': confidence_threshold
    }

print('📊 Evaluating Cross-lingual Search Performance...')
results = evaluate_search_accuracy(search_engine, multilingual_data, confidence_threshold=0.7)

print(f'\n📈 Evaluation Results:')
print(f'  Intent-based Accuracy: {results["accuracy"]:.2%}')
print(f'  Correct matches: {results["correct_matches"]}/{results["total_queries"]}')
print(f'  High confidence matches (≥{results["confidence_threshold"]}): {results["high_confidence_matches"]}')
print(f'\n📊 Information Retrieval Metrics:')
print(f'  Precision@1: {results["precision_at_1"]:.2%}')
print(f'  Precision@3: {results["precision_at_3"]:.2%}')
print(f'  Recall@5: {results["recall_at_5"]:.2%}')
print(f'  Mean Reciprocal Rank: {results["mrr"]:.4f}')
print(f'\n🔧 API Usage:')
print(f'  Total API requests: {nova_embeddings.request_count}')

print('\n✅ Evaluation complete!')

## Summary

You've built a complete multilingual search system that understands meaning across language boundaries. The system converts text in any language into numerical vectors that capture semantic content, stores these vectors in a managed database, and performs fast similarity searches to find relevant content regardless of the query language.

Nova Multimodal Embeddings handles the complex task of creating cross-lingual representations, while S3 Vectors provides scalable storage without requiring you to manage database servers. The modular architecture you built works with both small test datasets and production-scale data, with clean interfaces and proper error handling throughout.

Performance-wise, S3 Vectors costs significantly less than running your own vector database infrastructure, scales automatically to handle millions of vectors, and returns search results in milliseconds. 

To scale this for production, you could expand to the full MASSIVE dataset with all 52 languages, add multimodal capabilities by including images and documents alongside text, or set up real-time data pipelines for dynamic content updates. The foundation you've built handles the core challenges of multilingual semantic search and provides a solid base for enterprise applications.

For monitoring and optimization, track your search quality metrics over time, watch API costs and optimize batch sizes for embedding generation, and set up alerts for performance issues. S3 Vectors automatically encrypts your data and integrates with IAM for access control, while CloudTrail provides audit logging for compliance requirements.

This architecture gives you a production-ready multilingual search system that can scale with your needs while keeping operational complexity minimal.